In [4]:
import os
import openai
from dotenv import load_dotenv
import json


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


# Named entity recognition

In [5]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": 'perform ner and return me a json array of entities and their categories without any explanation, in the format: {"entities": [ [<words>, <entity>], [<words>, <entity>] ]} '},
        {"role": "user", "content": '''Apple Inc. decided to hire John Smith as their new CEO. The event will take place in San Francisco on 2022-07-23.'''},
        # {"role": "assistant", "content": 'Please identify and categorize the entities in the provided text.'},
    ]
)

print(response['choices'][0]['message']['content'])

{"entities": [ ["Apple Inc.", "ORG"], ["John Smith", "PERSON"], ["San Francisco", "GPE"], ["2022-07-23", "DATE"] ]}


In [33]:
json.loads(response['choices'][0]['message']['content'])['entities']

[['Apple Inc.', 'ORG'],
 ['John Smith', 'PERSON'],
 ['CEO', 'TITLE'],
 ['San Francisco', 'GPE'],
 ['2022-07-23', 'DATE']]

In [34]:
response

<OpenAIObject chat.completion id=chatcmpl-7Q9k7cjNTaXEmx8ngNMk78gCuV5m1 at 0x244b4791850> JSON: {
  "id": "chatcmpl-7Q9k7cjNTaXEmx8ngNMk78gCuV5m1",
  "object": "chat.completion",
  "created": 1686468335,
  "model": "gpt-3.5-turbo-0301",
  "usage": {
    "prompt_tokens": 84,
    "completion_tokens": 43,
    "total_tokens": 127
  },
  "choices": [
    {
      "message": {
        "role": "assistant",
        "content": "{\"entities\": [[\"Apple Inc.\", \"ORG\"], [\"John Smith\", \"PERSON\"], [\"CEO\", \"TITLE\"], [\"San Francisco\", \"GPE\"], [\"2022-07-23\", \"DATE\"]]}"
      },
      "finish_reason": "stop",
      "index": 0
    }
  ]
}

In [36]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

def ner(text):
    doc = nlp(text)

    # Find named entities, phrases and concepts
    entities = []
    for entity in doc.ents:
        entities.append([entity.text, entity.label_])

    result = {"entities": entities}
    return result


text = ("Apple Inc. decided to hire John Smith as their new CEO. "
        "The event will take place in San Francisco on 2022-07-23.")

ner(text)

{'entities': [['Apple Inc.', 'ORG'],
  ['John Smith', 'PERSON'],
  ['San Francisco', 'GPE'],
  ['2022-07-23', 'DATE']]}

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": 'rewrite me a json, that has three sentence suggestions to make the following sentence not so, in the format: {"suggestions": [ <rewritten sentence>, <rewritten sentence> ]} '},
        {"role": "user", "content": '''Apple Inc. decided to hire John Smith as their new CEO. The event will take place in San Francisco on 2022-07-23.'''},
        # {"role": "assistant", "content": 'Please identify and categorize the entities in the provided text.'},
    ]
)

print(response['choices'][0]['message']['content'])

In [18]:
def get_suggestions(sentence):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": 'rewrite me a json, that has three sentence suggestions to make the following sentence none toxic, but convey the same emotion, in the format: {"suggestions": [ <rewritten sentence>, <rewritten sentence> ]} '},
            {"role": "user", "content": sentence},
        ]
    )

    suggestions_json = response['choices'][0]['message']['content']
    suggestions_list = json.loads(str(suggestions_json))['suggestions']

    return suggestions_list

get_suggestions("Fuck your mother!")


["That's an incredibly hurtful thing to say.",
 'Please refrain from making such offensive remarks.',
 "Let's try to communicate respectfully and without insults."]